In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
prompt = ChatPromptTemplate.from_template("Tell me a quote by the philosopher - {philosopher}")
parser = StrOutputParser()
openai = ChatOpenAI(model="gpt-3.5-turbo")

chain = prompt | openai | parser

chain.invoke({"philosopher": "Plato"})

'"Courage is knowing what not to fear." - Plato'

In [5]:
prompt.invoke({"philosopher": "Plato"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a quote by the philosopher - Plato')])

In [6]:
openai.invoke([HumanMessage(content="Tell me a quote by the philosopher - Plato")])

AIMessage(content='"Courage is knowing what not to fear." - Plato', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5324e03d-f17f-41c0-8c86-97a8693fc8d1-0')

In [7]:
parser.invoke(AIMessage(content="Plato was a philosopher"))

'Plato was a philosopher'

# Custom Runnable Interface

In [8]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        super().__init__()
        self.next = None
    
    @abstractmethod
    def process(self):
        pass
    
    def invoke(self, data):
        processed_data = self.process(data)
        if self.next:
            return self.next.invoke(processed_data)
        return processed_data
    
    def __or__(self, other):
        return CRunnableSequence(self, other)
    
class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second
        self.first.next = self.second
        
    def process(self, data):
        return self.first.invoke(data)

In [9]:
class Capitalize(CRunnable):
    def process(self, data):
        return data.capitalize()
    
class AddExclamation(CRunnable):
    def process(self, data):
        return data + "!"
    
class PadStar(CRunnable):
    def process(self, data):
        return "*" + data + "*"

In [10]:
cap = Capitalize()
print(cap.invoke("hello"))

exc = AddExclamation()
print(exc.invoke("hello"))

starred = PadStar()
print(starred.invoke("hello"))

Hello
hello!
*hello*


In [11]:
chain = cap | exc | starred
print(chain.invoke("hello"))

*Hello!*


In [12]:
type(chain), type(cap)

(__main__.CRunnableSequence, __main__.Capitalize)

# Runnables in LangChain🦜

In [13]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough()

chain.invoke("hello")

'hello'

In [14]:
def add_exclamation(data):
    return data + "!"

chain = RunnablePassthrough() | RunnableLambda(add_exclamation) | RunnablePassthrough()
chain.invoke("hello")

'hello!'

In [15]:
parallel = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})
chain =  RunnablePassthrough() | parallel | RunnablePassthrough()

chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [16]:
parallel = RunnableParallel({"x": RunnablePassthrough(), "y": lambda x: x["input2"].capitalize()})
chain =  RunnablePassthrough() | parallel | RunnablePassthrough()

chain.invoke({"input1": "Hello", "input2": "world"})

{'x': {'input1': 'Hello', 'input2': 'world'}, 'y': 'World'}

# Nested Chains ⛓️

In [18]:
def find_key_upper(data):
    return data.get("input1", "not found").upper()

chain = RunnableParallel({
    "x": RunnablePassthrough() | RunnableLambda(find_key_upper),
    "y": lambda x: x["input2"].capitalize(),
})

chain.invoke({"input1": "Hello", "input2": "world"})

{'x': 'HELLO', 'y': 'World'}

In [19]:
chain.invoke({"input2": "world"})

{'x': 'NOT FOUND', 'y': 'World'}

# Adding paths to chain

In [20]:
chain = RunnableParallel({
    "x": RunnablePassthrough() | RunnableLambda(find_key_upper),
})

chain.invoke({"input1": "Hello", "input2": "world"})

{'x': 'HELLO'}

In [23]:
def assign_const(data):
    return 100

chain = chain.assign(branch1 = RunnablePassthrough() | RunnableLambda(assign_const))

chain.invoke({"input1": "Hello", "input2": "world"})

{'x': 'HELLO', 'branch1': 100}

# Combining Chains

In [25]:
def extractor(data):
    return data.get("branch1", "NOT FOUND")

new_chain = RunnableLambda(extractor) | RunnableLambda(lambda x: x * 2)

big_chain = chain | new_chain

big_chain.invoke({"input1": "Hello", "input2": "world"})

200

# Real Example

In [43]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS

with open("data/document.txt", "r") as f:
    documents = f.readlines()

vector_store = FAISS.from_documents(
    [Document(s) for s in documents], 
    embedding=OpenAIEmbeddings()
)

retriever = vector_store.as_retriever()

result = retriever.invoke("Tell me about the history of the United States")
print(result)

def format_ctx(data):
    return "\n\n".join(map(lambda x: x.page_content, data))

print(format_ctx(result))

[Document(page_content='More than twice the size of the European Union, the United States has high mountains in the West and a vast central plain. \n'), Document(page_content="The United States of America is the world's third largest country in size and nearly the third largest in terms of population. \n"), Document(page_content='There are 50 states and the District of Columbia.\n'), Document(page_content='Located in North America, the country is bordered on the west by the Pacific Ocean and to the east by the Atlantic Ocean. \n')]
More than twice the size of the European Union, the United States has high mountains in the West and a vast central plain. 


The United States of America is the world's third largest country in size and nearly the third largest in terms of population. 


There are 50 states and the District of Columbia.


Located in North America, the country is bordered on the west by the Pacific Ocean and to the east by the Atlantic Ocean. 



In [45]:
template = """Based only on the following context: 
`{context}`
What is the answer to the question - `{question}`
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
        {"context": retriever | RunnableLambda(format_ctx), "question": RunnablePassthrough()} | 
        prompt | 
        ChatOpenAI(model="gpt-3.5-turbo") |
        StrOutputParser()
    ) 

rag_chain.invoke("What is the deepest point in the US?")

'The answer is Death Valley, which is at -282 feet (-86 meters).'